In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
telemetry = pd.read_csv("./Motorq Data Science Assignment/telemetry_data.csv")
triggers = pd.read_csv("./Motorq Data Science Assignment/triggers_soc.csv")
pnid = pd.read_csv("./Motorq Data Science Assignment/vehicle_pnid_mapping.csv")
arti_ign = pd.read_json("./Motorq Data Science Assignment/artificial_ign_off_data.json")

In [3]:
telemetry["TIMESTAMP"] = pd.to_datetime(telemetry["TIMESTAMP"], format="mixed")

In [4]:
telemetry

,Unnamed: 0,VEHICLE_ID,TIMESTAMP,SPEED,IGNITION_STATUS,EV_BATTERY_LEVEL,ODOMETER
0,0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:45:42.018,NaN,NaN,NaN,NaN
1,1,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:58:08.017,NaN,NaN,NaN,NaN
2,2,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:57:02.031,NaN,NaN,NaN,38685.9
3,3,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:38:57.002,0.0,on,NaN,NaN
4,4,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 02:00:05.004,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1823547,749582,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 03:11:40.400,NaN,NaN,NaN,NaN
1823548,749583,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 03:11:41.014,NaN,NaN,NaN,NaN
1823549,749584,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 03:11:42.500,NaN,NaN,NaN,32072.7
1823550,749585,ef67f6d6-196a-4bc0-94fa-c13ad9672dac,2023-01-21 03:12:00.021,NaN,NaN,57.6,NaN


In [5]:
triggers["CTS"] = pd.to_datetime(triggers["CTS"], format="mixed")

In [6]:
triggers["PNID"] = triggers["PNID"].apply(str)

In [7]:
triggers

,Unnamed: 0,CTS,PNID,NAME,VAL
0,0,2021-12-01 17:58:27-05:30,206958332,IGN_CYL,OFF
1,1,2021-12-01 17:58:27-05:30,206958332,IGN_CYL,OFF
2,2,2021-12-01 17:56:24-05:30,206958332,IGN_CYL,ON
3,3,2021-12-01 17:56:24-05:30,206958332,CHARGE_STATE,50.1
4,4,2021-12-01 17:56:24-05:30,206958332,IGN_CYL,ON
...,...,...,...,...,...
68665,68665,2022-01-28 20:45:07-05:30,193290358,IGN_CYL,OFF
68666,68666,2022-01-28 20:45:52-05:30,207925262,IGN_CYL,ON
68667,68667,2022-01-28 20:45:52-05:30,207925262,CHARGE_STATE,49.0
68668,68668,2022-01-28 20:45:52-05:30,207925262,IGN_CYL,ON


In [8]:
triggers.isna().sum()

Unnamed: 0    0
CTS           0
PNID          0
NAME          0
VAL           0
dtype: int64

In [9]:
pnid

,Unnamed: 0,ID,IDS
0,0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,"[\n ""256932585"",\n ""256932577"",\n ""25693258..."
1,1,ae912623-e122-4bfb-b2a1-7d407ea42b1b,NaN
2,2,39424065-b3be-45e5-8f7d-b7f16a6c190a,NaN
3,3,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,"[\n ""270199389"",\n ""270199393"",\n ""27019938..."
4,0,9893c80d-f282-46cf-a794-280f0306c936,"[\n ""256933093"",\n ""256933081"",\n ""25693308..."
5,1,56d8ca94-9b18-41d1-831f-7afd905326d4,"[\n ""331750017"",\n ""331750025"",\n ""33175002..."
6,2,026af092-a01e-4b0a-af38-f459725dabc6,NaN
7,3,654f3d0f-f2f9-4fc6-856d-91337bef92f7,NaN
8,4,ae912623-e122-4bfb-b2a1-7d407ea42b1b,NaN
9,5,39424065-b3be-45e5-8f7d-b7f16a6c190a,NaN


In [10]:
pnid.columns

Index(['Unnamed: 0', 'ID', 'IDS'], dtype='object')

In [11]:
arti_ign["timestamp"] = pd.to_datetime(arti_ign["timestamp"], format="mixed")

In [12]:
arti_ign

,vehicleId,timestamp,type
0,3e937810-3a3b-48d6-80ad-56ae12c3bed8,2022-01-31 19:13:06.001000+00:00,artificial_event
1,3e937810-3a3b-48d6-80ad-56ae12c3bed8,2022-01-31 04:25:05.011000+00:00,artificial_event
2,460a67ab-ee58-4279-b96d-ac56ac029d3e,2022-01-30 01:58:57.004000+00:00,artificial_event
3,9893c80d-f282-46cf-a794-280f0306c936,2022-01-30 21:43:42.027000+00:00,artificial_event
4,fb20e847-d2a1-4583-b874-393c9d1f3db9,2022-01-31 04:25:05.038000+00:00,artificial_event
...,...,...,...
406,f0704100-56e6-4117-ae4f-cf97e5d2d132,2021-12-19 07:32:14.004000+00:00,artificial_event
407,fb20e847-d2a1-4583-b874-393c9d1f3db9,2021-12-19 05:54:05.037000+00:00,artificial_event
408,fb20e847-d2a1-4583-b874-393c9d1f3db9,2021-12-19 04:10:51.041000+00:00,artificial_event
409,460a67ab-ee58-4279-b96d-ac56ac029d3e,2021-12-18 22:45:17.030000+00:00,artificial_event


In [13]:
arti_ign.isna().sum()

vehicleId    0
timestamp    0
type         0
dtype: int64

In [14]:
pnid2 = pd.DataFrame({
    "sno":[],
    "ID":[],
    "PNID":[]
})
for i in range(pnid.shape[0]):
    ins = pnid.iloc[i]
    ids = ins["IDS"]
    if isinstance(ids, str):
        ids = list(ids.replace("\n","").replace("[","").replace("]","").replace("\"","").split(","))
        ids = [s.strip() for s in ids]
        for id in ids:
            pnid2.loc[len(pnid2.index)] = [ins["Unnamed: 0"],ins['ID'], id]
    else:
        pnid2.loc[len(pnid2.index)] =  [ins["Unnamed: 0"],ins['ID'], np.NaN]
pnid2

,sno,ID,PNID
0,0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256932585
1,0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256932577
2,0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256932581
3,0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256932589
4,0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,256782425
...,...,...,...
160,14,04105a12-59b9-447b-865f-599f48eed1d7,364272761
161,14,04105a12-59b9-447b-865f-599f48eed1d7,278608654
162,14,04105a12-59b9-447b-865f-599f48eed1d7,364272749
163,14,04105a12-59b9-447b-865f-599f48eed1d7,364272753


In [15]:
pnid2.isna().sum()

sno     0
ID      0
PNID    6
dtype: int64

In [16]:
pnid2.drop_duplicates(subset="PNID", inplace=True)

In [17]:
pnid2["PNID"].duplicated().value_counts()

PNID
False    155
Name: count, dtype: int64

In [18]:
triggers = pd.merge(left=triggers, right=pnid2[["ID", "PNID"]], on="PNID", how="inner")
triggers

,Unnamed: 0,CTS,PNID,NAME,VAL,ID
0,6,2021-12-01 19:26:58-05:30,256832791,IGN_CYL,ON,9893c80d-f282-46cf-a794-280f0306c936
1,7,2021-12-01 19:27:01-05:30,256832791,CHARGE_STATE,73.3,9893c80d-f282-46cf-a794-280f0306c936
2,404,2021-12-08 23:52:25-05:30,256832791,IGN_CYL,OFF,9893c80d-f282-46cf-a794-280f0306c936
3,405,2021-12-08 23:52:32-05:30,256832791,CHARGE_STATE,47.4,9893c80d-f282-46cf-a794-280f0306c936
4,424,2021-12-14 22:42:28-05:30,256832791,IGN_CYL,ON,9893c80d-f282-46cf-a794-280f0306c936
...,...,...,...,...,...,...
23418,68481,2022-01-28 21:42:27-05:30,270129825,CHARGE_STATE,51.7,fc86bd41-dd55-4d15-bb3f-35e31c7759e6
23419,68530,2022-01-28 21:40:02-05:30,270129825,IGN_CYL,OFF,fc86bd41-dd55-4d15-bb3f-35e31c7759e6
23420,68531,2022-01-28 21:40:04-05:30,270129825,CHARGE_STATE,52.1,fc86bd41-dd55-4d15-bb3f-35e31c7759e6
23421,68532,2022-01-28 21:40:02-05:30,270129825,IGN_CYL,OFF,fc86bd41-dd55-4d15-bb3f-35e31c7759e6


In [19]:
triggers["ID"].value_counts()

ID
66bd55df-eaf0-49c8-b9e1-7759b85e9325    7523
9893c80d-f282-46cf-a794-280f0306c936    5660
fc86bd41-dd55-4d15-bb3f-35e31c7759e6    5253
d44b765e-d86f-46f8-ba46-8cd913c49285    4987
Name: count, dtype: int64

In [20]:
ignition_events = pd.DataFrame({
    "VEHICLE_ID":[],
    "TIMESTAMP":[],
    "event_name":[],
})

In [24]:
prev = dict()
for i in range(telemetry.shape[0]):
    tel = telemetry.iloc[i]
    if not prev.__contains__(tel["VEHICLE_ID"]):
        prev[tel["VEHICLE_ID"]]=tel['IGNITION_STATUS']
    if tel['IGNITION_STATUS']=="on" and prev[tel["VEHICLE_ID"]]=="off":
        ignition_events.loc[len(ignition_events.index)] = [tel["VEHICLE_ID"], tel["TIMESTAMP"], "ignitionOn"]
        previous_status = "on"
    elif tel['IGNITION_STATUS']=="off" and prev[tel["VEHICLE_ID"]]=="on":
        ignition_events.loc[len(ignition_events.index)] = [tel["VEHICLE_ID"], tel["TIMESTAMP"], "ignitionOff"]
        previous_status = "off"


In [25]:
prev = dict()
for i in range(triggers.shape[0]):
    tel = triggers.iloc[i]
    if tel["NAME"]!="IGN_CYL":continue
    if not prev.__contains__(tel["ID"]):
        prev[tel["ID"]]=tel['VAL']
    if tel['VAL']=="ON" and prev[tel["ID"]]=="OFF":
        ignition_events.loc[len(ignition_events.index)] = [tel["ID"], tel["CTS"], "ignitionOn"]
        previous_status = "ON"
    elif tel['VAL']=="OFF" and prev[tel["ID"]]=="ON":
        ignition_events.loc[len(ignition_events.index)] = [tel["ID"], tel["CTS"], "ignitionOff"]
        previous_status = "OFF"

In [26]:
for i in range(arti_ign.shape[0]):
    x = arti_ign.iloc[i]
    ignition_events.loc[len(ignition_events.index)] = [x["vehicleId"], x["timestamp"], "ignitionOff"]

In [27]:
ignition_events

,VEHICLE_ID,TIMESTAMP,event_name
0,56d8ca94-9b18-41d1-831f-7afd905326d4,2023-01-06 00:03:30.029000,ignitionOff
1,56d8ca94-9b18-41d1-831f-7afd905326d4,2023-01-06 00:03:30.100000,ignitionOff
2,8ce8b281-3c5e-4b17-a863-630b9fc813dc,2023-01-06 00:21:57.100000,ignitionOff
3,8ce8b281-3c5e-4b17-a863-630b9fc813dc,2023-01-06 00:21:57.011000,ignitionOff
4,9eb5ac45-bbf9-4a07-83d8-044372f336c7,2023-01-05 22:51:17.011000,ignitionOff
...,...,...,...
7156,f0704100-56e6-4117-ae4f-cf97e5d2d132,2021-12-19 07:32:14.004000+00:00,ignitionOff
7157,fb20e847-d2a1-4583-b874-393c9d1f3db9,2021-12-19 05:54:05.037000+00:00,ignitionOff
7158,fb20e847-d2a1-4583-b874-393c9d1f3db9,2021-12-19 04:10:51.041000+00:00,ignitionOff
7159,460a67ab-ee58-4279-b96d-ac56ac029d3e,2021-12-18 22:45:17.030000+00:00,ignitionOff


In [ ]:
triggers["VAL"].value_counts()

VAL
OFF         15692
ON          15188
Active       3607
Aborted      2401
Complete      415
            ...  
4.3             6
3.1             4
5.0             3
1.9             3
0.7             1
Name: count, Length: 261, dtype: int64

In [ ]:
ev_triggers = pd.DataFrame({
    "VEHICLE_ID":[],
    "TIMESTAMP":[],
    "event_name":[]
})

In [ ]:
for i in range(triggers.shape[0]):
    tel = triggers.iloc[i]
    if tel["NAME"]!="EV_CHARGE_STATE":continue
    if tel['VAL']=="Active":
        ev_triggers.loc[len(ev_triggers.index)] = [tel["ID"], tel["CTS"], "Active"]
    elif tel['VAL']=="Aborted":
        ev_triggers.loc[len(ev_triggers.index)] = [tel["ID"], tel["CTS"], "Aborted"]
    elif tel['VAL']=="Complete":
        ev_triggers.loc[len(ev_triggers.index)] = [tel["ID"], tel["CTS"], "Complete"]

In [ ]:
ev_triggers

,VEHICLE_ID,TIMESTAMP,event_name
0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-12-01 19:57:32-05:30,Active
1,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-12-06 22:43:33-05:30,Aborted
2,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-12-07 01:49:27-05:30,Aborted
3,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-12-07 01:49:57-05:30,Aborted
4,NaN,2021-12-07 01:58:59-05:30,Aborted
...,...,...,...
6418,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2022-01-28 21:09:02-05:30,Aborted
6419,NaN,2022-01-28 20:30:37-05:30,Aborted
6420,NaN,2022-01-28 20:30:37-05:30,Aborted
6421,NaN,2022-01-28 20:30:37-05:30,Aborted


In [ ]:
combined = pd.concat([ignition_events, ev_triggers])
combined

,VEHICLE_ID,TIMESTAMP,event_name
0,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 01:38:57.002000,ignitionOn
1,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 04:18:05.031000,ignitionOff
2,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-09-23 04:16:31.002000,ignitionOn
3,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-10-03 02:51:53.035000,ignitionOff
4,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2021-10-03 03:02:52.029000,ignitionOn
...,...,...,...
6418,66bd55df-eaf0-49c8-b9e1-7759b85e9325,2022-01-28 21:09:02-05:30,Aborted
6419,NaN,2022-01-28 20:30:37-05:30,Aborted
6420,NaN,2022-01-28 20:30:37-05:30,Aborted
6421,NaN,2022-01-28 20:30:37-05:30,Aborted


In [ ]:
eventref = pd.DataFrame({
    "VEHICLE_ID":[],
    "TIMESTAMP":[],
    "event_name":[],
    "pnid":[],
    
    "battery_level":[]
})
eventref

,VEHICLE_ID,TIMESTAMP,event_name,pnid,battery_level


In [ ]:
telemetry = telemetry.sort_values(by=["VEHICLE_ID", "TIMESTAMP"])
telemetry

,Unnamed: 0,VEHICLE_ID,TIMESTAMP,SPEED,IGNITION_STATUS,EV_BATTERY_LEVEL,ODOMETER
1415308,341343,026af092-a01e-4b0a-af38-f459725dabc6,2023-01-01 02:36:17.442,30.0,NaN,98.0,35439.0
1415309,341344,026af092-a01e-4b0a-af38-f459725dabc6,2023-01-01 02:36:47.885,NaN,NaN,98.0,35439.3
1415310,341345,026af092-a01e-4b0a-af38-f459725dabc6,2023-01-01 02:36:47.886,48.0,NaN,NaN,NaN
1415311,341346,026af092-a01e-4b0a-af38-f459725dabc6,2023-01-01 02:37:18.561,6.0,NaN,98.0,35439.5
1188880,114915,026af092-a01e-4b0a-af38-f459725dabc6,2023-01-01 02:37:48.866,0.0,NaN,97.0,35439.5
...,...,...,...,...,...,...,...
1460034,386069,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2023-01-30 22:53:26.096,NaN,NaN,NaN,NaN
1460035,386070,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2023-01-30 22:53:28.185,NaN,off,NaN,NaN
1460036,386071,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2023-01-30 22:53:31.187,NaN,NaN,NaN,NaN
1460037,386072,fc86bd41-dd55-4d15-bb3f-35e31c7759e6,2023-01-30 22:53:41.194,NaN,NaN,72.9,NaN


In [ ]:
triggers = triggers.sort_values(by=["PNID", "CTS"])
triggers

,Unnamed: 0,CTS,PNID,NAME,VAL,ID
52902,52902,2021-09-04 05:31:16-05:30,186776160,IGN_CYL,ON,NaN
52903,52903,2021-09-04 05:31:18-05:30,186776160,CHARGE_STATE,89.0,NaN
52884,52884,2021-09-04 05:45:40-05:30,186776160,IGN_CYL,OFF,NaN
52885,52885,2021-09-04 05:45:43-05:30,186776160,CHARGE_STATE,89.0,NaN
52886,52886,2021-09-04 05:53:16-05:30,186776160,IGN_CYL,ON,NaN
...,...,...,...,...,...,...
21098,21098,2022-01-31 01:13:09-05:30,270768911,CHARGE_STATE,54.1,NaN
21133,21133,2022-01-31 01:23:47-05:30,270768911,IGN_CYL,OFF,NaN
21135,21135,2022-01-31 01:23:47-05:30,270768911,IGN_CYL,OFF,NaN
21134,21134,2022-01-31 01:23:52-05:30,270768911,CHARGE_STATE,52.5,NaN


In [ ]:
previous_status = "off"
for i in range(telemetry.shape[0]):
    tel = telemetry.iloc[i]
    if tel['IGNITION_STATUS']=="on" and previous_status=="off":
        eventref.loc[len(eventref.index)] = [tel["VEHICLE_ID"], tel["TIMESTAMP"], "ignitionOn", np.NaN, tel["EV_BATTERY_LEVEL"]]
        previous_status = "on"
    elif tel['IGNITION_STATUS']=="off" and previous_status=="on":
        eventref.loc[len(eventref.index)] = [tel["VEHICLE_ID"], tel["TIMESTAMP"], "ignitionOff", np.NaN, tel["EV_BATTERY_LEVEL"]]
        previous_status = "off"

In [ ]:
previous_status = "OFF"
for i in range(triggers.shape[0]):
    tel = triggers.iloc[i]
    if tel["NAME"]!="IGN_CYL":continue
    if tel['VAL']=="ON" and previous_status=="OFF":
        eventref.loc[len(eventref.index)] = [tel["ID"], tel["CTS"], "ignitionOn", tel["PNID"], np.NaN]
        previous_status = "ON"
    elif tel['VAL']=="OFF" and previous_status=="ON":
        eventref.loc[len(eventref.index)] = [tel["ID"], tel["CTS"], "ignitionOff", tel["PNID"], np.NaN]
        previous_status = "OFF"

In [ ]:
for i in range(arti_ign.shape[0]):
    x = arti_ign.iloc[i]
    eventref.loc[len(eventref.index)] = [x["vehicleId"], x["timestamp"], "ignitionOff", np.NaN, np.NaN]

In [ ]:
for i in range(triggers.shape[0]):
    tel = triggers.iloc[i]
    if tel["NAME"]!="EV_CHARGE_STATE":continue
    if tel['VAL']=="Active":
        eventref.loc[len(eventref.index)] = [tel["ID"], tel["CTS"], "Active", tel["PNID"], np.NaN]
    elif tel['VAL']=="Aborted":
        eventref.loc[len(eventref.index)] = [tel["ID"], tel["CTS"], "Aborted", tel["PNID"], np.NaN]
    elif tel['VAL']=="Complete":
        eventref.loc[len(eventref.index)] = [tel["ID"], tel["CTS"], "Complete", tel["PNID"], np.NaN]

In [ ]:
eventref

,VEHICLE_ID,TIMESTAMP,event_name,pnid,battery_level
0,04105a12-59b9-447b-865f-599f48eed1d7,2023-01-06 18:57:56.002000,ignitionOn,NaN,NaN
1,04105a12-59b9-447b-865f-599f48eed1d7,2023-01-06 19:00:28.013000,ignitionOff,NaN,NaN
2,04105a12-59b9-447b-865f-599f48eed1d7,2023-01-13 17:25:45.002000,ignitionOn,NaN,NaN
3,04105a12-59b9-447b-865f-599f48eed1d7,2023-01-13 17:27:47.078000,ignitionOff,NaN,NaN
4,3bb48ce7-bb7e-4b4f-b902-b0561e1cd2e0,2023-01-30 21:24:49.002000,ignitionOn,NaN,NaN
...,...,...,...,...,...
39867,NaN,2022-01-29 07:01:13-05:30,Active,270768911,NaN
39868,NaN,2022-01-29 23:54:59-05:30,Active,270768911,NaN
39869,NaN,2022-01-29 23:54:59-05:30,Active,270768911,NaN
39870,NaN,2022-01-29 23:55:30-05:30,Aborted,270768911,NaN


In [ ]:
for i in range(eventref.shape[0]):
    tel = eventref
    # if tel["battery_level"]!=np.NaN: continue
    ts = tel["TIMESTAMP"]
    if np.all(tel["VEHICLE_ID"] != np.NaN):
        ind = telemetry[(telemetry["TIMESTAMP"]==ts) & (telemetry["VEHICLE_ID"] == tel["VEHICLE_ID"])].index
        j=1
        l = telemetry.shape[0]
        while(ind-j>0 and ind+j<l):
            x = telemetry.iloc[ind-j]
            y = telemetry.iloc[ind+j]
            if np.all(x["VEHICLE_ID"] != tel["VEHICLE_ID"]) and np.all(y["VEHICLE_ID"]!= tel["VEHICLE_ID"]):break
            m=0
            minus = None
            if x["VEHICLE_ID"] == tel["VEHICLE_ID"]:
                m = abs(x["TIMESTAMP"] - tel["TIMESTAMP"])
                minus = True 
            if y["VEHICLE_ID"] == tel["VEHICLE_ID"]:
                b = abs(x["TIMESTAMP"] - tel["TIMESTAMP"])
                if b<m:
                    m = b
                    minus = False
            if m < pd.to_timedelta('300 s'):
                if minus:
                    bl = telemetry.iloc[ind-j]["EV_BATTERY_LEVEL"]
                    if bl!=np.NaN:
                        eventref.iloc[i]["battery_level"] = bl
                        break
                else:
                    bl = telemetry.iloc[ind+j]["EV_BATTERY_LEVEL"]
                    if bl!=np.NaN:
                        eventref.iloc[i]["battery_level"] = bl
                        break
                




ValueError: Can only compare identically-labeled Series objects

In [ ]:
eventref

,VEHICLE_ID,TIMESTAMP,event_name,pnid,battery_level
0,04105a12-59b9-447b-865f-599f48eed1d7,2023-01-06 18:57:56.002000,ignitionOn,NaN,NaN
1,04105a12-59b9-447b-865f-599f48eed1d7,2023-01-06 19:00:28.013000,ignitionOff,NaN,NaN
2,04105a12-59b9-447b-865f-599f48eed1d7,2023-01-13 17:25:45.002000,ignitionOn,NaN,NaN
3,04105a12-59b9-447b-865f-599f48eed1d7,2023-01-13 17:27:47.078000,ignitionOff,NaN,NaN
4,3bb48ce7-bb7e-4b4f-b902-b0561e1cd2e0,2023-01-30 21:24:49.002000,ignitionOn,NaN,NaN
...,...,...,...,...,...
39867,NaN,2022-01-29 07:01:13-05:30,Active,270768911,NaN
39868,NaN,2022-01-29 23:54:59-05:30,Active,270768911,NaN
39869,NaN,2022-01-29 23:54:59-05:30,Active,270768911,NaN
39870,NaN,2022-01-29 23:55:30-05:30,Aborted,270768911,NaN
